# CMPE 452 - ASSIGNMENT 1

Perceptron scratch algoithim and the tensorflow keras is included in this file. Please run the first block for each of the algorithims to get the training and test files imported into the code. 



In [2]:
# Import statements - Run for both keras and python based code
import numpy as np
from numpy import genfromtxt
import os
import math as math
np.genfromtxt

#on your computer choose the files  to train and test - make sure that they have names iris_test.txt, iris_train.txt (Select both of them at the same time)
from google.colab import files
upload = files.upload()


Saving iris_test.txt to iris_test.txt
Saving iris_train.txt to iris_train.txt


In [0]:

#Function used to split the data from inputs and outputs
def importDataAndSplit():
  #Splitting data
  irisTestX = genfromtxt('iris_test.txt', delimiter= ",", usecols=[0,1,2,3]) #take the first 4 columns for input for testing
  irisTestY = genfromtxt('iris_test.txt', delimiter= ",", usecols=[4], dtype=str) # take last column for output for testing

  irisTrainX = genfromtxt('iris_train.txt', delimiter= ",", usecols=[0,1,2,3]) #take the first 4 columns for input for training
  irisTrainY = genfromtxt('iris_train.txt', delimiter= ",", usecols=[4], dtype=str) # take last column for output for training
  
  #Add a -l column at the end of each dataset as a bias input for both training and testing
  negTrain = np.ones((irisTrainX.shape[0],1))
  negTrain = negTrain*-1
  negTest = np.ones((irisTestX.shape[0],1))
  negTest = negTest*-1
  irisTrainX = np.concatenate( [ irisTrainX, negTrain ] , axis = 1)
  irisTestX = np.concatenate( [ irisTestX, negTest ] , axis = 1)
  
  
  #converting from string to int for the classification of the flowers for both training and testing
  # 0 - Iris-setosa
  # 1 - Iris-versicolor
  # 2 - Iris-virgina
  for i in range(irisTrainY.shape[0]):
    if irisTrainY[i] == "Iris-setosa":
      irisTrainY[i] = "0"
    elif irisTrainY[i] == "Iris-versicolor":
      irisTrainY[i] = "1"
    else:
      irisTrainY[i] = "2"

  for i in range(irisTestY.shape[0]):
    if irisTestY[i] == "Iris-setosa":
      irisTestY[i] = "0"
    elif irisTestY[i] == "Iris-versicolor":
      irisTestY[i] = "1"
    else:
      irisTestY[i] = "2"

  irisTrainY = irisTrainY.astype(np.int)
  irisTestY = irisTestY.astype(np.int)
  
  #return the 4 data sets 
  return irisTestX, irisTestY, irisTrainX, irisTrainY




In [0]:
class Perceptron(object):

    #define the attirubutes for the function
    def __init__(self, no_of_inputs, itera=1000, c=0.01):
        self.itera = itera #interation
        self.c = c #learning rate
        self.weights = np.random.rand(no_of_inputs + 1, 3) #weights as a 2D array (15)
        self.longestrun = 0 #for the pocket alogorithim - store the longest run
        self.currentrun = 0 #for the pocket algorithim - store the current run
        self.best_weights =  np.copy(self.weights) #for the pocket algorithim - to put the best found weights
        
    #Used to predict which perceptron/neuron has fired - takes the inputs to find output
    def predict(self, inputs):
        
        #take the dot product which muliples the weights with associated inputs 
        summation = np.dot(inputs,self.best_weights)#returns a 1*3 matric
        max1 = -100
        spot = -1
        #find the highest value and return the location
        for r in range(3):
          if summation[r] > max1:
            max1 = summation[r]
            spot = r
        return spot 
      
    #Used to train the network - takes the training input and output
    def train(self, training_inputs, training_outputs):
        #for the amount of iterations we want to 
        for k in range(self.itera):
            error = 0
            #for all the data sets in training 
            for inputs, output in zip(training_inputs, training_outputs):
                
                #forward propogation
                z2 = np.dot(inputs,self.weights)
                #apply sigmoid function so between 1 and 0
                a2 = sigmoid(z2) #1*3 matrix
                  
                #find which neuron of the 3 should be activated
                target = np.zeros(3)
                target[output] = 1
                
                
                for i in range(3):
                  error += (target[i] - a2[i]) ** 2
                
                #find the error using the target - predicated value from the sigmoid function
                change = []
                for i in range(3):
                  x = (target[i] - a2[i])
                  change.append(x)
                
                if np.argmax(a2) == np.argmax(target): #if equal increase run length
                    self.currentrun += 1
                    if self.currentrun > self.longestrun:   #if longestrun is less than or equal to current run
                      self.longestrun = self.currentrun
                      self.best_weights = np.copy(self.weights) #copy current weights into best weights
                else:
                    self.currentrun = 0 #set curent run to 0 and apply weight changes
                    for y in range(4):
                      for x in range(3):
                        self.weights[y][x] += self.c * (change[x] * inputs[y]) 
              
            if error == 0:
               self.best_weights = np.copy(self.weights)
               break
             
    # Used to test the neural network - accepts the x or input values of the test data
    def test(self, test_input):
        p = []
        print (self.best_weights)
        #for every data set
        for inputs in test_input:
          #call the perdict function to find which flower it is and append to list
          predication = self.predict(inputs)
          p.append(predication)
        return p #returns a list of predicated values
    
                           
      

In [0]:
#Function for sigmoid function
def sigmoid(inX):
    r = 1.0/(1+ np.exp(-inX))
    return r

#Function to output data into a test file
def output_data_txt(inputs, predictions):
    #puts outputd file on google drive - need to give it access
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)
    
    #Cange the int classification of flowers back to string
    pred = np.array(predictions)
    p = [[0 for x in range(30)] for y in range(1)]
    for i in range (30):
      if pred[i] == 0.0:
        p[0][i] = "Iris-setosa"
      elif pred[i] == 1.0:
        p[0][i] = "Iris-versicolor"
      else:
        p[0][i] = "Iris-virginica" 
    p = np.array(p, dtype=str)
    
    #shape the array so now it it vertical
    p = np.reshape(p, (30, 1))
    #delete the bais input column
    inputs = np.delete(inputs, 4, 1) #remove -1 (bias input) column
    #combine the 2 arrays together - inputs and predictions
    final =  np.concatenate( [ inputs, p ] , axis = 1)
    #save to a text file
    with open('/content/gdrive/My Drive/Output/output.txt', 'w') as f:
      np.savetxt(f, (final), fmt = '%s')
      
  

In [39]:
#call the perceptron class and init function - paramters: number of inputs to each perceptron (not including bias)
per = Perceptron(4)
#call function to split train and test data into 4 lists
test_x, test_y, train_x, train_y = importDataAndSplit()
#train neural network with training data
per.train(train_x, train_y)
#use test data to see how accurate it is
perd = per.test(test_x)

# To see how accurate the neural network is comapred to the correct values
acc = 0.0
for i in range(30):
    if perd[i] == int(test_y[i]):
        acc += 1
print(acc / 30 * 100, "%")

# Output information into a text file
output_data_txt(test_x, perd)

[[-0.00173001  0.09418506 -0.85209047]
 [ 0.92680948  0.1534393  -0.64316556]
 [-1.38284192 -0.05207456  1.08696319]
 [ 0.10197962 -0.0886286   1.13190831]
 [ 0.37731776  0.76604002  0.30366055]]
90.0 %


KeyboardInterrupt: ignored

Part 2 - Implementing using tensorflow and keras

Please run the first bloack to import the lists into the algorithim.

In [40]:
# Import statements - also run the import block found at the top of code
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import to_categorical

#Separate the x and y data for each the training and testing set
test_x = genfromtxt('iris_test.txt', delimiter= ",", usecols=[0,1,2,3]) #take the first 4 columns for input for testing
test_y = genfromtxt('iris_test.txt', delimiter= ",", usecols=[4], dtype=str) # take last column for output for testing

train_x = genfromtxt('iris_train.txt', delimiter= ",", usecols=[0,1,2,3]) #take the first 4 columns for input for training
train_y = genfromtxt('iris_train.txt', delimiter= ",", usecols=[4], dtype=str)

#for the y data 
for i in range(train_y.shape[0]):
    if train_y[i] == "Iris-setosa":
      train_y[i] = "0"
    elif train_y[i] == "Iris-versicolor":
      train_y[i] = "1"
    else:
      train_y[i] = "2"

for i in range(test_y.shape[0]):
    if test_y[i] == "Iris-setosa":
      test_y[i] = "0"
    elif test_y[i] == "Iris-versicolor":
      test_y[i] = "1"
    else:
      test_y[i] = "2"

test_y = test_y.astype(np.int)
train_y = train_y.astype(np.int)

"""
Training the model
"""
#changes it to a binary persentation 
train_y = to_categorical(train_y) 

#build model - using a stack layer model - inputs -> [3 hidden nodes] -> 3 outputs
m = tf.keras.Sequential()
m.add(Dense(3, input_dim=4, activation='sigmoid')) #	4 inputs connected to a hidden layer with the activation function sigmoid
m.add(Dense(3, activation='softmax')) # output layer with an activation of the softmax function so final result is between 1 and 0

#define loss and otptimzer functions
m.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

#train the model
m.fit(train_x, train_y, epochs=550, batch_size=32)

"""
Testing model
"""
#get prediction of test data
cl = m.predict_classes(test_x, batch_size=32)

#Print predictions and overall accuracy of the netowrk by comparing it to the true output
import numpy as np
print("Prediction :")
print(cl)
print("Target :")
print(test_y)
print ("Accuracy: ")
acc = 0.0
for i in range(30):
    if cl[i] == int(test_y[i]):
        acc += 1
print(acc / 30 * 100, "%")

Epoch 1/550
120/120 [==============================] - 0s 651us/sample - loss: 0.2354 - acc: 0.3333
Epoch 2/550
120/120 [==============================] - 0s 123us/sample - loss: 0.2345 - acc: 0.3333
Epoch 3/550
120/120 [==============================] - 0s 119us/sample - loss: 0.2337 - acc: 0.3333
Epoch 4/550
120/120 [==============================] - 0s 126us/sample - loss: 0.2330 - acc: 0.3333
Epoch 5/550
120/120 [==============================] - 0s 126us/sample - loss: 0.2323 - acc: 0.3333
Epoch 6/550
120/120 [==============================] - 0s 123us/sample - loss: 0.2315 - acc: 0.3333
Epoch 7/550
120/120 [==============================] - 0s 125us/sample - loss: 0.2308 - acc: 0.3333
Epoch 8/550
120/120 [==============================] - 0s 126us/sample - loss: 0.2301 - acc: 0.3333
Epoch 9/550
120/120 [==============================] - 0s 175us/sample - loss: 0.2295 - acc: 0.3333
Epoch 10/550
120/120 [==============================] - 0s 110us/sample - loss: 0.2288 - acc: 0.3333